# RTT and Window size effects on TCP
<i>Adapted for use with FABRIC from [Effect of RTT and Window Size on TCP](https://www.cs.unc.edu/Research/geni/geniEdu/04-TcpDelayWinSize.html)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

In this notebook module you will be able to experiment how RTT and TCP window size affect the throughput, you will learn how to adjust the RTT by adjusting time delay and you will experiment with the window size using the iperf commands.
This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Setup the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

####  Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [ ]:
slice_name = 'RttWindowTCP'
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

In [ ]:
# ADDDED Subnet
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("10.1.1.0/24")
available_ips = list(subnet)[1:]


network_name = 'LAN_CONN'

In [ ]:
# configure nodes
try:
    for node in slice.get_nodes():
        node_iface = node.get_interface(network_name=network_name)  
        node_addr = available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_addr, subnet=subnet)
        
        print(f'### Ip adress for node {node.get_name()} is {node_addr} ###')
        
        stdout, stderr = node.execute(f'sudo ip link set dev enp7s0 up')
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')

except Exception as e:
    print(f"Fail: {e}")

### 1.3 Instalation of tools for the tutorial
This includes "ifconfig", "iperf", and Rstudio for the graphics in the assignment

In [ ]:
# configure nodes
try:
    for node in slice.get_nodes():        
        node.execute("sudo apt update")
        node.execute("sudo apt install net-tools")
        node.execute("sudo apt install iperf")

except Exception as e:
    print(f"Fail: {e}")

In [ ]:
try:
    for node in slice.get_nodes():
        #Graphics
        node.upload_file("./scripts/RT-data-analysis.R","RT-data-analysis.R")
        node.upload_file("./scripts/RT-output.sh","RT-output.sh")
        node.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x RT-output.sh ")
except Exception as e:
    print(f"Fail: {e}")

    

## 2. Run Experiment
### 2.1 Adjusting the MTU (Maximum Transfer Unit)
In this step we will setup the MTU to use later in the tutorial
1. Login to the each node and keep a terminal open on each node
2. Run `ifconfig` on the terminal Server
   <br> you should take a note on the interface that contains the server's assigned ip address (10.1.1.1), The interface is of the form ```enp#s0``` where '#' is some number, this will be the interface that is able to connect the server and the client.
3. Since this is a GRE tunnel, you can adjust the MTU size for the interfaces, this can be done with the following command
	<br> `sudo ifconfig <interface name> mtu #` The MTU can be adjusted as needed but for this experiment we will use `# = 1400`
4. repeat steps 2 and 3 on the client node, remember that it is likely that the interfaces are the same but it is not guaranteed



### 2.2 Adding and Removing Delay
<img src="./figures/RTT.png">

In this step we will find how to change the delay of transfer.
1. on the Server terminal run the following steps
2. First we need to know what the approximate base RTT (Round Trip Time) between the nodes is, we are able to find this information using the ping tool: `ping <Client ip addr> -c 10` we will ping 10 times to find an average RTT in the report. alternatively you can get an estimate by looking at the individual pings
3. We can use a Linux tool called tc (traffic control) to add a fixed amount of delay to the packets leaving the interface.
	<br> In this case the delay will be 100ms using the following command
	<br>`sudo tc qdisc add dev <interface name> root netem delay 100ms`
	<br> The command translates to: adjust tc queuing discipline (qdisc) by adding (add) a network emulation (netem) delay of 100 milliseconds to the device at the root
4. If we try running the ping again `ping <Client ip addr> -c 10`
	<br> we should now see that the RTT average is now 100ms longer
5. we are also able to remove the delay using `sudo tc qdisc del dev <interface name> root`
6. Pinging the client should now be the same as it was before adding the delay `ping <Client ip addr> -c 10`


### 2.3 Adjusting The TCP window size
<br><img src="./figures/WindowSize.png"><br>
In this step we will learn how to adjust the window size and how it affects the performance
1. iperf is a tool for measuring TCP and UDP bandwidth performance. This tool allows us to change the window size. In the Server Terminal run the following command: `iperf -s`
	<br> This node will run iperf in server mode "-s" and it allows it to receive iperf traffic
2. On the Client terminal, start the ipref client using `iperf -c <server ip addr> -t 10`
	<br> This command starts iperf in client mode "-c" and opens a TCP connection to the server, the client will begin sending packets using the default window size. The client will stop after 10 seconds "-t 10".
3. Once the Client is done take note of the average bandwidth and use "Ctrl-c" in the Server terminal to kill the server
4. Now we can repeat the procedure but we will limit the window size on both the client and the server using the following command: `iperf -s -w 2KB`
	<br> Note that Linux automatically doubles the window size set by iperf.
5. In the client terminal type: `iperf -c <server ip addr> -t 10 -w 2KB` to start the connection
	<br> Once the client is done you can compare the average bandwidth value and notice that it is significantly less than when we used the default window size.

## Continue to The Assignment notebook

Once You have completed this notebook you should be able to continue to the RTT Window size Assignment Notebook, you can ether open it on the explorer or click [Here](./Assignment.ipynb) to open the next notebook.